<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/02-Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables


In [ ]:
!pip install -q openai==1.93.0 cohere==5.15.0 tiktoken==0.8.0 google-genai==1.23.0

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"

from google.colab import userdata

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] =  userdata.get('GOOGLE_API_KEY')

In [ ]:
# False: Generate the embedding for the dataset. (Associated cost with using OpenAI endpoint)
# True: Load the dataset that already has the embedding vectors.
load_embedding = False

# Load Dataset


## Download Dataset (JSON)


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.


In [ ]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv

--2026-01-22 05:18:43--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.002s  

2026-01-22 05:18:44 (81.3 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]

--2026-01-22 05:18:44--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP r

## Read File


In [ ]:
# Split the input text into chunks of specified size.
def split_into_chunks(text, chunk_size=1024):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i : i + chunk_size])

    return chunks

In [ ]:
import csv

chunks = []

# Load the file as a CSV
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        chunks.extend(split_into_chunks(row[1]))

In [ ]:
import pandas as pd

# Convert the JSON list to a Pandas Dataframe
df = pd.DataFrame(chunks, columns=["chunk"])

df.keys()

Index(['chunk'], dtype='object')

# Generate Embedding


In [ ]:
from openai import OpenAI

client = OpenAI()


# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
    try:
        # Remove newlines
        text = text.replace("\n", " ")
        res = client.embeddings.create(input=[text], model="text-embedding-3-small")

        return res.data[0].embedding

    except:
        return None

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

# Generate embedding
if not load_embedding:
    print("Generating embeddings...")
    embeddings = []
    for index, row in tqdm(df.iterrows()):
        # df.at[index, 'embedding'] = get_embedding( row['chunk'] )
        embeddings.append(get_embedding(row["chunk"]))

    embeddings_values = pd.Series(embeddings)
    df.insert(loc=1, column="embedding", value=embeddings_values)

# Or, load the embedding from the file.
else:
    print("Loaded the embedding file.")
    # Load the file as a CSV
    df = pd.read_csv("mini-llama-articles-with_embeddings.csv")
    # Convert embedding column to an array
    df["embedding"] = df["embedding"].apply(lambda x: np.array(eval(x)), 0)

Generating embeddings...


0it [00:00, ?it/s]

In [ ]:
# df.to_csv('mini-llama-articles-with_embeddings.csv')

# User Question


In [ ]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding(QUESTION)

len(QUESTION_emb)

1536

# Test Cosine Similarity


Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.


In [ ]:
BAD_SOURCE_emb = get_embedding("The sky is blue.")
GOOD_SOURCE_emb = get_embedding("LLaMA2 model has a total of 2B parameters.")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity([QUESTION_emb], [BAD_SOURCE_emb]))
print("> Good Response Score:", cosine_similarity([QUESTION_emb], [GOOD_SOURCE_emb]))

> Bad Response Score: [[0.02584474]]
> Good Response Score: [[0.83150482]]


# Calculate Cosine Similarities


In [ ]:
# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity([QUESTION_emb], df["embedding"].tolist())

print(cosine_similarities)

[[0.46760729 0.46905013 0.25972739 0.29367753 0.31959666 0.40156336
  0.41498416 0.45246161 0.45926015 0.12601315 0.11748975 0.01352324
  0.22600041 0.21419775 0.10129437 0.33057305 0.10740327 0.34687857
  0.16303888 0.08730439 0.3481973  0.22837149 0.19200548 0.26469503
  0.24924009 0.34813181 0.24829939 0.32757415 0.41437236 0.41329412
  0.46356805 0.38337796 0.46842242 0.35637118 0.35391393 0.30134742
  0.29934453 0.29252605 0.40026729 0.46472325 0.39470932 0.41037076
  0.44694265 0.43165486 0.35902346 0.33972972 0.51342821 0.20928596
  0.4019752  0.3282994  0.42825111 0.48261479 0.45029543 0.34246237
  0.32077422 0.42585515 0.24601999 0.18092139 0.23650172 0.34269227
  0.34377983 0.20478407 0.19755017 0.22440721 0.21109169 0.42294993
  0.26373248 0.3043026  0.33602069 0.38360973 0.23528777 0.2434589
  0.37079358 0.28025918 0.49042748 0.53041652 0.37816339 0.43769764
  0.3774359  0.39256165 0.30081991 0.41710517 0.46740307 0.45410368
  0.35159718 0.21227624 0.42614263 0.31599625 0.4

In [ ]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax(cosine_similarities)

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print(indices)

[114  75  89]


In [ ]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate(df.chunk[indices]):
    print(f"> Chunk {idx+1}")
    print(item)
    print("----")

> Chunk 1
by Meta that ventures into both the AI and academic spaces. The model aims to help researchers, scientists, and engineers advance their work in exploring AI applications. It will be released under a non-commercial license to prevent misuse, and access will be granted to academic researchers, individuals, and organizations affiliated with the government, civil society, academia, and industry research facilities on a selective case-by-case basis. The sharing of codes and weights allows other researchers to test new approaches in LLMs. The LLaMA models have a range of 7 billion to 65 billion parameters. LLaMA-65B can be compared to DeepMind's Chinchilla and Google's PaLM. Publicly available unlabeled data was used to train these models, and training smaller foundational models require less computing power and resources. LLaMA 65B and 33B have been trained on 1.4 trillion tokens in 20 different languages, and according to the Facebook Artificial Intelligence Research (FAIR) team,

## Gemini client function

In [ ]:
from google import genai
from google.genai import types

def gemini_response(system_prompt,prompt):
    client = genai.Client()

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
            system_instruction=system_prompt
        ),
    )

    try:
        return response.text
    except Exception as e:
        print(f"An error occurred: {e}")

# Augment the Prompt


In [ ]:
from google import genai
from google.genai import types

# Use the Gemini API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = """You are an assistant and expert in answering questions from a chunks of content.
                      Only answer AI-related question, else say that you cannot answer this question."""

    # User prompt with the user's question
    prompt = """
        Read the following informations that might contain the context you require to answer the question.
        You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag.
        Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        Please provide an informative and accurate answer to the following question based on the avaiable context.
        Be concise and take your time. \nQuestion: {}\nAnswer:"
      """

    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format("".join(df.chunk[indices]), QUESTION)


    response = gemini_response(system_prompt,prompt)
    print(response)

except Exception as e:
    print(f"An error occurred: {e}")


Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited.


## How Augmenting the Prompt can address knowledge cutoff limitations and hallucinations

In [ ]:
# Consider this as a retrieved chunk
# https://ai.meta.com/blog/llama-4-multimodal-intelligence/

Example_chunk = """
# Llama 4 Technical Overview

Meta has introduced the Llama 4 model family, marking a significant advancement in open-weight multimodal AI with three distinct variants designed for different use cases. The Llama 4 Scout represents the more compact option with 17 billion active parameters utilizing 16 experts within a total parameter count of 109 billion, designed to fit on a single H100 GPU with Int4 quantization. The Llama 4 Maverick serves as the flagship model with 17 billion active parameters but employs 128 experts across 400 billion total parameters, fitting on a single H100 host and designed as the primary workhorse for general assistant and chat applications. The preview model Llama 4 Behemoth functions as a teacher model with 288 billion active parameters, 16 experts, and nearly 2 trillion total parameters, currently still in training but demonstrating state-of-the-art performance on mathematical and reasoning benchmarks.
The architecture represents Meta's first implementation of mixture-of-experts (MoE) design in the Llama series, fundamentally changing how the models process information by activating only a fraction of total parameters for each token. The MoE architecture alternates between dense and mixture-of-experts layers, with the Maverick model specifically using 128 routed experts alongside a shared expert, where each token is processed by the shared expert and routed to one of the 128 specialized experts. This design significantly improves compute efficiency during both training and inference while delivering higher quality outputs compared to dense models with equivalent computational budgets.
Native multimodality represents another breakthrough, achieved through early fusion architecture that seamlessly integrates text and vision tokens into a unified model backbone during pre-training. This approach enables joint training with massive amounts of unlabeled text, image, and video data, with the vision encoder based on MetaCLIP but trained separately in conjunction with a frozen Llama model to optimize adaptation to the language model. The models support up to 48 images during pre-training and demonstrate strong performance with up to 8 images in post-training scenarios, enabling sophisticated visual reasoning and understanding tasks across multiple input modalities.
The training infrastructure incorporates several technical innovations, including the MetaP technique for reliable hyperparameter optimization that transfers well across different model configurations and training scenarios. The models were trained on over 30 trillion tokens, representing more than double the training data of Llama 3, with support for 200 languages including over 100 languages with more than 1 billion tokens each. Training efficiency was maximized through FP8 precision without quality sacrifice, achieving 390 TFLOPs per GPU utilization on 32,000 GPUs during Behemoth pre-training, while a specialized mid-training phase enhanced core capabilities and enabled the industry-leading 10 million token context length for Scout.
Context length capabilities represent a dramatic advancement, with Llama 4 Scout supporting 10 million tokens compared to Llama 3's 128K limit through the innovative iRoPE architecture that employs interleaved attention layers without positional embeddings and inference-time temperature scaling of attention. This architecture enables superior length generalization and opens possibilities for multi-document summarization, extensive user activity parsing for personalization, and reasoning over vast codebases, with compelling performance demonstrated in retrieval tasks and cumulative negative log-likelihood evaluations over 10 million tokens of code.
The post-training pipeline underwent significant revision with a three-stage approach consisting of lightweight supervised fine-tuning, online reinforcement learning, and lightweight direct preference optimization. A critical insight involved removing over 50% of data tagged as easy using Llama models as judges, focusing training on harder examples to prevent over-constraining that could restrict exploration during online RL and lead to suboptimal performance in reasoning, coding, and mathematics. The continuous online RL strategy alternated between model training and adaptive data filtering to retain only medium-to-hard difficulty prompts, proving highly beneficial for compute and accuracy trade-offs.
Safety and bias mitigation received comprehensive attention through multi-layered approaches spanning pre-training data filtering, post-training policy conformance, and system-level safeguards. Meta introduced Generative Offensive Agent Testing (GOAT) to address traditional red-teaming limitations by simulating multi-turn interactions of medium-skilled adversarial actors, enabling more efficient vulnerability detection while allowing human experts to focus on novel adversarial areas. Particular emphasis was placed on addressing political and social bias inherent in internet training data, with Llama 4 demonstrating significant improvements over Llama 3 in presenting balanced perspectives on contentious issues without favoring particular viewpoints.
The distillation process for creating smaller models from Behemoth required novel approaches, including a dynamic loss function that weighted soft and hard targets throughout training, with codistillation during pre-training amortizing computational costs for the majority of training data. Post-training the 2 trillion parameter Behemoth model necessitated pruning 95% of supervised fine-tuning data compared to 50% for smaller models, followed by large-scale reinforcement learning focused on sampling hard prompts through pass@k analysis and constructing training curricula of increasing difficulty. The unprecedented scale required complete infrastructure overhaul, including optimized MoE parallelization and fully asynchronous online RL training framework that improved training efficiency by approximately 10x over previous generations through flexible GPU allocation and resource balancing across multiple models.
"""

In [ ]:
QUESTION = "How many parameters LLaMA 4 model has?"

# Formulating the system prompt
system_prompt = """ You are an assistant and expert in answering questions from a chunks of content.
                    Only answer AI-related question, else say that you cannot answer this question."""

# Combining the system prompt with the user's question
prompt = """Read the following informations that might contain the context you require to answer the question.
            You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag.
            Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n".
            Please provide an informative and accurate answer to the following question based on the avaiable context.
            Be concise and take your time. \nQuestion: {}\nAnswer:
            """

prompt = prompt.format(Example_chunk, QUESTION)

response = gemini_response(system_prompt,prompt)
print(response)

The Llama 4 model family has three distinct variants with different parameter counts:

*   **Llama 4 Scout:** 17 billion active parameters with a total parameter count of 109 billion.
*   **Llama 4 Maverick:** 17 billion active parameters across 400 billion total parameters.
*   **Llama 4 Behemoth:** 288 billion active parameters and nearly 2 trillion total parameters.


# Without Augmentation


In [ ]:
QUESTION = "How many parameters LLaMA 4 model has?"

# Formulating the system prompt
system_prompt = "You are an assistant and expert in answering questions."

# Combining the system prompt with the user's question
prompt = "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"

prompt = prompt.format(QUESTION)

response = gemini_response(system_prompt,prompt)
print(response)

The LLaMA 4 model does not exist. The latest model in the LLaMA series is Llama 3.
